#  Тестовое задание для стражировки в Avito  
## Автор: Павленко Арсений 
## Часть 2. Предобработка данных

In [1]:
from os.path import join

In [2]:
import pandas as pd
import numpy as np

In [3]:
import re

In [4]:
from string import punctuation
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
import pymorphy2 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
PATH_TO_DATA = "../../data/01_Avito_dataset/"

In [7]:
PATH_TO_SAVE_PREPR_TRAIN = "../02_Files/Prepocessed_data/train/"
PATH_TO_SAVE_PREPR_TEST = "../02_Files/Prepocessed_data/test/"

In [8]:
PATH_TO_SAVE_TTS_TRAIN = "../02_Files/Train_test_split/train/"
PATH_TO_SAVE_TTS_VALID = "../02_Files/Train_test_split/valid"
PATH_TO_SAVE_TTS_TEST = "../02_Files/Train_test_split/test/"

# Загрузка данных

In [9]:
train_df = pd.read_csv(join(PATH_TO_DATA,'train.csv'), index_col='item_id')

test_df = pd.read_csv(join(PATH_TO_DATA,'test.csv'), index_col='item_id')

In [10]:
train_df.head()

,title,description,price,category_id
item_id,,,,
0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [11]:
test_df.head()

,title,description,price
item_id,,,
489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0


# Предобработка текста

Для улучшения качества классификации проведем предобработку текста, содержащегося в признакая `title` и  `desscription`.

Реализуем функцию `preprocess` на вход которой будет поступать текст который мы хотим предобработать.

Данная функци будет совершать следующие преобразования:
- Приведение всех слов к нижнему регистру;
- Удаление специальных символов;
- Замена буквы 'ё' на 'е';
- разбиение текста на токены;
- нормализация текста, в данном случае Лемматизация, то есть приведение слова к нормальной форме на основе словаря (реализованную в библиотеке pymorphy2.


In [12]:
morph = pymorphy2.MorphAnalyzer()

In [13]:
specSymb = punctuation 
pattern = re.compile("[" + re.escape(specSymb) + "]")

In [14]:
stopRus = stopwords.words('russian')
stopEng = stopwords.words('english')

In [15]:
def preprocess(text):
    text = text.lower()
    text = re.sub(pattern, '', text)
    text = text.replace('ё', 'е')
    
    tokens = WhitespaceTokenizer().tokenize(text)
    
    
    filtered_words = [word for word in tokens if word not in stopRus]
    filtered_words = [word for word in tokens if word not in stopEng]
    
    lemm_words = [morph.parse(word)[0].normal_form for word in tokens]

    return " ".join(lemm_words)

In [16]:
%%time
train_df["title"] = train_df["title"].apply(preprocess)
train_df['description'] = train_df['description'].apply(preprocess)

CPU times: user 1h 4min 4s, sys: 1.97 s, total: 1h 4min 6s
Wall time: 1h 4min 7s


In [17]:
%%time
test_df["title"] = test_df["title"].apply(preprocess)
test_df['description'] = test_df['description'].apply(preprocess)

CPU times: user 31min 42s, sys: 780 ms, total: 31min 43s
Wall time: 31min 43s


In [18]:
train_df.head()

,title,description,price,category_id
item_id,,,,
0,картина,гобелен размер 139х84см,1000.0,19
1,стул из прессовать кожа,продать недорого 4 стул из светлый прессовать ...,1250.0,22
2,домашний минь баня,минь баня мб1минь сауна предназначить для прин...,13000.0,37
3,эксклюзивный коллекция книга трансаэро подарок,продать эксклюзивный коллекция книга выпустить...,4000.0,43
4,ноутбук aser,продаваться ноутбук acer e5511c2ta купить в ко...,19000.0,1


In [19]:
test_df.head()

,title,description,price
item_id,,,
489517,стоик журнальный сталь,продать журнальный столик изготавливать стол и...,10000.0
489518,iphone 5 64gb,телефон в хороший состояние комплект гарантия ...,12500.0
489519,утеплитель,теплопеленлидер тепло толщина утеплитель 20 мм...,250.0
489520,пальто демисезонный,продать пальто женский букле в отличный состоя...,1700.0
489521,samsung syncmaster t200n,условно рабочий проблема в панель настройка мо...,1000.0


Так как предобработка текста занимает много времени, сохраним полученные данные.

In [20]:
train_df.to_csv(join(PATH_TO_SAVE_PREPR_TRAIN, 'train.csv'))
test_df.to_csv(join(PATH_TO_SAVE_PREPR_TEST, 'test.csv'))

# Разбиение на обучение и валидацию

Разобъем выборку на обучающую и валидационную и сохраним полученные результаты для дальнейшего использования

In [29]:
X = train_df[['title', 'description', 'price']]
y = pd.DataFrame(train_df['category_id'])

In [30]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.382, random_state=97)

Реализуем функцию `separation_df`, которая будет сохранять каждый признак полученных датфреймов в отдельный файл, так будет удобнее использовать их для дальнейшей обработки.

In [31]:
def separation_df(df, PATH_TO_DATA):
    features = df.columns.values
    for name in features:
        df[name].to_csv(join(PATH_TO_DATA,name + '.csv'), header=True)
    return 

In [33]:
separation_df(X_train, PATH_TO_SAVE_TTS_TRAIN)
separation_df(y_train, PATH_TO_SAVE_TTS_TRAIN)

separation_df(X_valid, PATH_TO_SAVE_TTS_VALID)
separation_df(y_valid, PATH_TO_SAVE_TTS_VALID)

separation_df(test_df, PATH_TO_SAVE_TTS_TEST)